# Topographical attributes extraction

Author: Thiago Nascimento (thiago.nascimento@eawag.ch)

This notebook is part of the EStreams publication and was used to extract and aggregate the topographical characteristics from the MERIT dataset.

* Note that this code enables not only the replicability of the current database but also the extrapolation to new catchment areas. 
* Additionally, the user should download and insert the original raw-data in the folder of the same name prior to run this code. 
* The original third-party data used were not made available in this repository due to redistribution and storage-space reasons.  

## Requirements
**Python:**

* Python>=3.6
* Jupyter
* geopandas=0.10.2
* numpy
* os
* pandas
* tqdm

Check the Github repository for an environment.yml (for conda environments) or requirements.txt (pip) file.

**Files:**

* data/topography/riv_pfaf_2_MERIT_Hydro_v07_Basins_v01.shp. Available at: https://www.reachhydro.org/home/params/merit-basins (Last access: 23 November 2023)
* data/shapefiles/estreams_catchments.shp
* data/gee/topography/EStreams_elevation_attributes_gee.csv. Elevation attributes CSV-file(s) exported from GEE.
* data/gee/EStreams_slope_attributes_gee.csv. Slope attributes CSV-file(s) exported from GEE.

**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References

* Yamazaki, D. et al. A high-accuracy map of global terrain elevations. Geophys Res Lett 44, 5844–5853 (2017).
* Yamazaki, D. et al. MERIT Hydro: A High-Resolution Global Hydrography Map Based on Latest Topography Dataset. Water Resour Res 55, 5053–5073 (2019).

## License
* MERIT: Dual-license - CC-BY-NC 4.0 & ODbL 1.0. http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_DEM/index.html (Last access: 27 November 2023)

## Observations
* This notebook assumes that the GEE code to export elevation and slope descriptors from the MERIT-dem dataset (EStreams_landscape_attributes_topography_gee.txt) was run before in the GEE platform and that the output CSV-files are locally available. 

# Import modules

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tqdm as tqdm
from utils.terrain import *

# Configurations

In [2]:
# Only editable variables:
# Relative path to your local directory
PATH = "../../.."

* #### The users should NOT change anything in the code below here.

In [3]:
# Non-editable variables:
PATH_OUTPUT = "results/staticattributes/"

# Set the directory:
os.chdir(PATH)

# Import data
## Catchment boundaries

In [4]:
catchment_boundaries = gpd.read_file('data/shapefiles/estreams_catchments.shp')
catchment_boundaries.head()

,id,area_km2,outlet_lat,outlet_lng,name,area_offic,layer,path,Code,basin_id,area_calc,geometry
0,FR003159,37,47.488,7.393,A100003001,38.6,FR003159,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003159,FR003159,37.183,"POLYGON ((7.30374 47.49375, 7.30708 47.49375, ..."
1,FR003160,227,47.626,7.239,A105003001,233.0,FR003160,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003160,FR003160,226.962,"POLYGON ((7.22291 47.63458, 7.22374 47.63458, ..."
2,FR003161,14,47.586,7.384,A106000101,15.0,FR003161,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003161,FR003161,13.595,"POLYGON ((7.38791 47.59041, 7.39874 47.59041, ..."
3,FR003162,70,47.622,7.275,A107020001,70.0,FR003162,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003162,FR003162,70.152,"POLYGON ((7.28375 47.60958, 7.28291 47.60958, ..."
4,FR003163,330,47.653,7.265,A108003001,325.0,FR003163,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003163,FR003163,330.158,"POLYGON ((7.22958 47.65291, 7.23208 47.65291, ..."


In [5]:
print("The total number of catchments to be processed are:", len(catchment_boundaries))

The total number of catchments to be processed are: 1972


## Merit-Hydro river network

In [6]:
river_net_EU_MERIT = gpd.read_file('data/topography/riv_pfaf_2_MERIT_Hydro_v07_Basins_v01.shp')
river_net_EU_MERIT

,COMID,lengthkm,lengthdir,sinuosity,slope,uparea,order,strmDrop_t,slope_taud,NextDownID,maxup,up1,up2,up3,up4,geometry
0,21000001,9.079822,5.620153,1.615583,0.000220,382.604819,2,2.0,0.000220,21000054,2,21000002,21000011,0,0,"LINESTRING (5.88917 47.94917, 5.88833 47.94917..."
1,21000002,2.842653,1.826319,1.556494,0.001124,325.410126,2,3.2,0.001124,21000001,2,21000004,21000013,0,0,"LINESTRING (5.90667 47.99833, 5.90750 47.99833..."
2,21000003,20.918003,14.119143,1.481535,0.001484,415.960268,2,31.1,0.001484,21000055,2,21000006,21000009,0,0,"LINESTRING (6.04750 47.95083, 6.04833 47.95083..."
3,21000004,12.775680,6.218682,2.054403,0.000609,275.794567,2,7.8,0.000609,21000002,2,21000005,21000016,0,0,"LINESTRING (5.91667 48.01333, 5.91750 48.01333..."
4,21000005,3.535374,2.233011,1.583232,0.001892,171.981367,2,6.7,0.001892,21000004,2,21000014,21000015,0,0,"LINESTRING (5.99083 48.03917, 5.99167 48.03833..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393459,29123032,30.204422,21.486454,1.405743,0.005751,82.869595,1,173.4,0.005751,0,0,0,0,0,0,"LINESTRING (43.49000 12.80583, 43.49083 12.805..."
393460,29123033,10.813198,8.296287,1.303378,0.014205,65.406667,1,153.2,0.014205,29122956,0,0,0,0,0,"LINESTRING (44.46250 12.82500, 44.46167 12.825..."
393461,29123034,16.082080,9.903307,1.623910,0.011262,67.604187,1,180.7,0.011262,0,0,0,0,0,0,"LINESTRING (44.59833 12.81833, 44.59750 12.819..."
393462,29123035,1.327687,0.656550,2.022219,0.000000,36.423539,1,0.0,0.000000,0,0,0,0,0,0,"LINESTRING (45.07917 12.90917, 45.07833 12.908..."


## GEE outputs

In [7]:
# Elevation descriptors
topograhpy_atrributes_gee_elevation = pd.read_csv("data/gee/topography/EStreams_elevation_attributes_gee.csv", index_col=1)
topograhpy_atrributes_gee_elevation.drop(["system:index", ".geo"], axis = 1, inplace = True)
topograhpy_atrributes_gee_elevation.columns = ["ele_mt_max", "ele_mt_mean", "ele_mt_min"]
topograhpy_atrributes_gee_elevation

,ele_mt_max,ele_mt_mean,ele_mt_min
basin_id,,,
FR004955,1135.321045,678.992017,45.542904
FR003790,1089.800537,991.632224,900.237549
FR004604,553.520691,354.839497,221.044052
FR004188,827.323730,751.394385,578.952087
FR004158,273.875092,229.101021,188.949173
...,...,...,...
FR003594,910.082520,174.485980,-14.584519
FR003595,910.082520,174.485980,-14.584519
FR003597,910.082520,174.065864,-14.584519


In [8]:
# Slope descriptors
topograhpy_atrributes_gee_slope = pd.read_csv("data/gee/topography/EStreams_slope_attributes_gee.csv", index_col=1)
topograhpy_atrributes_gee_slope.drop(["system:index", ".geo"], axis = 1, inplace = True)
topograhpy_atrributes_gee_slope.columns = ["flat_area_fra", "slp_dg_mean", "steep_area_fra"]
topograhpy_atrributes_gee_slope = topograhpy_atrributes_gee_slope[["slp_dg_mean", "flat_area_fra", "steep_area_fra"]]
topograhpy_atrributes_gee_slope

,slp_dg_mean,flat_area_fra,steep_area_fra
basin_id,,,
FR004955,21.269205,0.016571,0.755344
FR003790,6.767178,0.123199,0.007702
FR004604,10.492727,0.105379,0.251798
FR004188,7.388196,0.165942,0.098543
FR004158,4.770470,0.295362,0.000000
...,...,...,...
FR003594,2.638076,0.713486,0.005328
FR003595,2.638076,0.713486,0.005328
FR003597,2.639058,0.713603,0.005395


In [9]:
topograhpy_atrributes_df = pd.concat([topograhpy_atrributes_gee_elevation, topograhpy_atrributes_gee_slope], axis=1)
topograhpy_atrributes_df

,ele_mt_max,ele_mt_mean,ele_mt_min,slp_dg_mean,flat_area_fra,steep_area_fra
basin_id,,,,,,
FR004955,1135.321045,678.992017,45.542904,21.269205,0.016571,0.755344
FR003790,1089.800537,991.632224,900.237549,6.767178,0.123199,0.007702
FR004604,553.520691,354.839497,221.044052,10.492727,0.105379,0.251798
FR004188,827.323730,751.394385,578.952087,7.388196,0.165942,0.098543
FR004158,273.875092,229.101021,188.949173,4.770470,0.295362,0.000000
...,...,...,...,...,...,...
FR003594,910.082520,174.485980,-14.584519,2.638076,0.713486,0.005328
FR003595,910.082520,174.485980,-14.584519,2.638076,0.713486,0.005328
FR003597,910.082520,174.065864,-14.584519,2.639058,0.713603,0.005395


# Reproject to projected coordinates system

In [10]:
# Here you can check the crs of the datasets:
print("CRS of catchment_boundaries:", catchment_boundaries.crs)
print("CRS of river_net_EU_MERIT:", river_net_EU_MERIT.crs)

CRS of catchment_boundaries: epsg:4326
CRS of river_net_EU_MERIT: epsg:4326


In [11]:
# Define the target CRS to ETRS89 LAEA
target_crs = 'EPSG:3035'  # ETRS89 LAEA

# Reproject the GeoDataFrame to the target CRS
catchment_boundaries_reprojected = catchment_boundaries.to_crs(target_crs)
river_net_EU_MERIT_reprojected = river_net_EU_MERIT.to_crs(target_crs)

In [12]:
# Here you can check the new crs of the datasets:
print("CRS of catchment_boundaries:", catchment_boundaries_reprojected.crs)
print("CRS of river_net_EU_MERIT:", river_net_EU_MERIT_reprojected.crs)

CRS of catchment_boundaries: EPSG:3035
CRS of river_net_EU_MERIT: EPSG:3035


# Compute area in sqm

In [13]:
catchment_boundaries_reprojected["area_sqm"] = catchment_boundaries_reprojected.area
catchment_boundaries_reprojected.head()

,id,area_km2,outlet_lat,outlet_lng,name,area_offic,layer,path,Code,basin_id,area_calc,geometry,area_sqm
0,FR003159,37,47.488,7.393,A100003001,38.6,FR003159,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003159,FR003159,37.183,"POLYGON ((4117741.729 2712605.833, 4117993.388...",3.718341e+07
1,FR003160,227,47.626,7.239,A105003001,233.0,FR003160,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003160,FR003160,226.962,"POLYGON ((4112221.355 2728469.645, 4112283.720...",2.269618e+08
2,FR003161,14,47.586,7.384,A106000101,15.0,FR003161,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003161,FR003161,13.595,"POLYGON ((4124451.602 2723112.595, 4125266.127...",1.359480e+07
3,FR003162,70,47.622,7.275,A107020001,70.0,FR003162,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003162,FR003162,70.152,"POLYGON ((4116693.863 2725523.575, 4116630.714...",7.015184e+07
4,FR003163,330,47.653,7.265,A108003001,325.0,FR003163,C:/Users/nascimth/Documents/Thiago/Eawag/Pytho...,FR003163,FR003163,330.158,"POLYGON ((4112796.581 2730486.398, 4112984.361...",3.301579e+08


# Disssolve niver-network


In [14]:
river_net_EU_MERIT_dissolved = river_net_EU_MERIT_reprojected.dissolve()

# River network density

In [15]:
# Create a spatial index for the river network
sindex = river_net_EU_MERIT_reprojected.sindex

# Initialize a dictionary to store results
results = {}

# Iterate through each catchment
for catchment_id in tqdm.tqdm(catchment_boundaries_reprojected.basin_id):

    # Filter the selected catchment
    selected_boundary = catchment_boundaries_reprojected[catchment_boundaries_reprojected['basin_id'] == catchment_id]

    # Calculate the total length of lines within the selected catchment
    total_length = 0
    boundary_bounds = selected_boundary.total_bounds
    possible_matches_index = list(sindex.intersection(boundary_bounds))
    possible_matches = river_net_EU_MERIT_reprojected.iloc[possible_matches_index]
    
    for index, row in possible_matches.iterrows():
        if row['geometry'].intersects(selected_boundary.unary_union):
            total_length += row['geometry'].intersection(selected_boundary.unary_union).length

    # Store the result in the dictionary
    results[catchment_id] = total_length

# Convert the dictionary to a DataFrame
strm_dens_df = pd.DataFrame(list(results.items()), columns=['basin_id', 'totalnet_length_m'])
strm_dens_df.set_index("basin_id", inplace = True)
strm_dens_df

100%|███████████████████████████████████████| 1972/1972 [08:24<00:00,  3.91it/s]


,totalnet_length_m
basin_id,
FR003159,3001.776112
FR003160,29814.627656
FR003161,0.000000
FR003162,7411.284701
FR003163,44237.036614
...,...
HR000314,15352.131649
HR000315,71649.965483
HR000316,81677.286662


In [16]:
# Convert the dictionary to a DataFrame
strm_dens_df = pd.DataFrame(list(results.items()), columns=['basin_id', 'totalnet_length_m'])
strm_dens_df.set_index("basin_id", inplace = True)
strm_dens_df

,totalnet_length_m
basin_id,
FR003159,3001.776112
FR003160,29814.627656
FR003161,0.000000
FR003162,7411.284701
FR003163,44237.036614
...,...
HR000314,15352.131649
HR000315,71649.965483
HR000316,81677.286662


In [17]:
strm_dens_df["area"] = catchment_boundaries_reprojected.set_index("basin_id").area
strm_dens_df["strm_dens"] = strm_dens_df["totalnet_length_m"] / strm_dens_df["area"] 
strm_dens_df

,totalnet_length_m,area,strm_dens
basin_id,,,
FR003159,3001.776112,3.718341e+07,0.000081
FR003160,29814.627656,2.269618e+08,0.000131
FR003161,0.000000,1.359480e+07,0.000000
FR003162,7411.284701,7.015184e+07,0.000106
FR003163,44237.036614,3.301579e+08,0.000134
...,...,...,...
HR000314,15352.131649,1.354620e+08,0.000113
HR000315,71649.965483,4.578644e+08,0.000156
HR000316,81677.286662,5.143687e+08,0.000159


# Enlongation ratio

In [18]:
# Create a dataframe to process the computation:
topograhpy_atrributes_enlon_ratio = pd.DataFrame()
topograhpy_atrributes_enlon_ratio["basin_id"] = catchment_boundaries_reprojected.basin_id
topograhpy_atrributes_enlon_ratio["area"] = catchment_boundaries_reprojected.area

# Assuming gdf is your GeoDataFrame with a Polygon geometry column named 'geometry'
topograhpy_atrributes_enlon_ratio['x_dimns'], topograhpy_atrributes_enlon_ratio['y_dimns'], topograhpy_atrributes_enlon_ratio['length']  = calculate_dimensions(catchment_boundaries_reprojected['geometry'])
topograhpy_atrributes_enlon_ratio

/Users/thiagomedeirosdonascimento/anaconda3/envs/eawag/lib/python3.10/site-packages/geopandas/geoseries.py:624: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/thiagomedeirosdonascimento/anaconda3/envs/eawag/lib/python3.10/site-packages/geopandas/geoseries.py:624: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)
/Users/thiagomedeirosdonascimento/anaconda3/envs/eawag/lib/python3.10/site-packages/geopandas/geoseries.py:624: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if yo

,basin_id,area,x_dimns,y_dimns,length
0,FR003159,3.718341e+07,10751.517197,6256.519026,10751.517197
1,FR003160,2.269618e+08,30576.024248,28070.884300,30576.024248
2,FR003161,1.359480e+07,6601.733591,8034.189790,8034.189790
3,FR003162,7.015184e+07,16651.087908,18134.816079,18134.816079
4,FR003163,3.301579e+08,27874.044029,32621.687356,32621.687356
...,...,...,...,...,...
1967,HR000314,1.354620e+08,21408.487630,20388.031006,21408.487630
1968,HR000315,4.578644e+08,29316.103745,37933.692955,37933.692955
1969,HR000316,5.143687e+08,49680.810045,50414.240188,50414.240188
1970,HR000317,1.847325e+08,23164.802343,25750.028392,25750.028392


In [19]:
# Enlongation ratio computation:
topograhpy_atrributes_enlon_ratio['elon_ratio'] = topograhpy_atrributes_enlon_ratio.apply(calculate_elongation_ratio, axis=1)
topograhpy_atrributes_enlon_ratio.set_index("basin_id", inplace = True)
topograhpy_atrributes_enlon_ratio

,area,x_dimns,y_dimns,length,elon_ratio
basin_id,,,,,
FR003159,3.718341e+07,10751.517197,6256.519026,10751.517197,0.639970
FR003160,2.269618e+08,30576.024248,28070.884300,30576.024248,0.555969
FR003161,1.359480e+07,6601.733591,8034.189790,8034.189790,0.517845
FR003162,7.015184e+07,16651.087908,18134.816079,18134.816079,0.521148
FR003163,3.301579e+08,27874.044029,32621.687356,32621.687356,0.628506
...,...,...,...,...,...
HR000314,1.354620e+08,21408.487630,20388.031006,21408.487630,0.613448
HR000315,4.578644e+08,29316.103745,37933.692955,37933.692955,0.636500
HR000316,5.143687e+08,49680.810045,50414.240188,50414.240188,0.507620


# Final aggregation

In [20]:
# First we create an empty table data frame to assing the values to it
topograhpy_df = pd.DataFrame(index = catchment_boundaries_reprojected.basin_id)

# Now we proceed with the concatenation:
topograhpy_df = pd.concat([topograhpy_df, topograhpy_atrributes_df, topograhpy_atrributes_enlon_ratio.elon_ratio, 
                        strm_dens_df.strm_dens], axis=1)

topograhpy_df

,ele_mt_max,ele_mt_mean,ele_mt_min,slp_dg_mean,flat_area_fra,steep_area_fra,elon_ratio,strm_dens
basin_id,,,,,,,,
FR003159,814.298950,554.320580,405.828430,7.888049,0.192174,0.100639,0.639970,0.000081
FR003160,819.578796,440.218408,286.717194,4.524214,0.469834,0.033254,0.555969,0.000131
FR003161,482.235748,427.693740,365.201385,2.545104,0.743314,0.000000,0.517845,0.000000
FR003162,482.235748,374.400640,283.611328,3.108262,0.574431,0.000582,0.521148,0.000106
FR003163,819.578796,415.502666,264.622986,4.237762,0.480515,0.024076,0.628506,0.000134
...,...,...,...,...,...,...,...,...
HR000314,1438.161865,847.528916,286.406372,10.195803,0.149550,0.225576,0.613448,0.000113
HR000315,1438.161865,551.787650,45.238613,8.578416,0.232201,0.165546,0.636500,0.000156
HR000316,1438.161865,530.773736,45.238613,8.452457,0.231580,0.159314,0.507620,0.000159


In [21]:
# Here we sort the columns:
topograhpy_df = topograhpy_df.sort_index(axis=0)
topograhpy_df

,ele_mt_max,ele_mt_mean,ele_mt_min,slp_dg_mean,flat_area_fra,steep_area_fra,elon_ratio,strm_dens
basin_id,,,,,,,,
FR003159,814.298950,554.320580,405.828430,7.888049,0.192174,0.100639,0.639970,0.000081
FR003160,819.578796,440.218408,286.717194,4.524214,0.469834,0.033254,0.555969,0.000131
FR003161,482.235748,427.693740,365.201385,2.545104,0.743314,0.000000,0.517845,0.000000
FR003162,482.235748,374.400640,283.611328,3.108262,0.574431,0.000582,0.521148,0.000106
FR003163,819.578796,415.502666,264.622986,4.237762,0.480515,0.024076,0.628506,0.000134
...,...,...,...,...,...,...,...,...
HR000313,1438.161865,709.957923,230.348175,10.308379,0.167417,0.237057,0.482943,0.000124
HR000314,1438.161865,847.528916,286.406372,10.195803,0.149550,0.225576,0.613448,0.000113
HR000315,1438.161865,551.787650,45.238613,8.578416,0.232201,0.165546,0.636500,0.000156


In [22]:
# Set the strm_dens to 1000km/km2 (improve units storage)
topograhpy_df.strm_dens = topograhpy_df.strm_dens*1000

In [23]:
# Assign the "basin_id" to the gauges names:
topograhpy_df.index.name = "basin_id"

In [24]:
# Convert from frac to perc:
topograhpy_df.flat_area_fra = topograhpy_df.flat_area_fra * 100
topograhpy_df.steep_area_fra = topograhpy_df.steep_area_fra * 100
topograhpy_df

,ele_mt_max,ele_mt_mean,ele_mt_min,slp_dg_mean,flat_area_fra,steep_area_fra,elon_ratio,strm_dens
basin_id,,,,,,,,
FR003159,814.298950,554.320580,405.828430,7.888049,19.217427,10.063933,0.639970,0.080729
FR003160,819.578796,440.218408,286.717194,4.524214,46.983366,3.325393,0.555969,0.131364
FR003161,482.235748,427.693740,365.201385,2.545104,74.331405,0.000000,0.517845,0.000000
FR003162,482.235748,374.400640,283.611328,3.108262,57.443051,0.058233,0.521148,0.105646
FR003163,819.578796,415.502666,264.622986,4.237762,48.051524,2.407575,0.628506,0.133988
...,...,...,...,...,...,...,...,...
HR000313,1438.161865,709.957923,230.348175,10.308379,16.741736,23.705744,0.482943,0.124326
HR000314,1438.161865,847.528916,286.406372,10.195803,14.954996,22.557650,0.613448,0.113332
HR000315,1438.161865,551.787650,45.238613,8.578416,23.220096,16.554570,0.636500,0.156487


In [25]:
# Round the data to 3 decimals
topograhpy_df = topograhpy_df.astype(float).round(3)
topograhpy_df

,ele_mt_max,ele_mt_mean,ele_mt_min,slp_dg_mean,flat_area_fra,steep_area_fra,elon_ratio,strm_dens
basin_id,,,,,,,,
FR003159,814.299,554.321,405.828,7.888,19.217,10.064,0.640,0.081
FR003160,819.579,440.218,286.717,4.524,46.983,3.325,0.556,0.131
FR003161,482.236,427.694,365.201,2.545,74.331,0.000,0.518,0.000
FR003162,482.236,374.401,283.611,3.108,57.443,0.058,0.521,0.106
FR003163,819.579,415.503,264.623,4.238,48.052,2.408,0.629,0.134
...,...,...,...,...,...,...,...,...
HR000313,1438.162,709.958,230.348,10.308,16.742,23.706,0.483,0.124
HR000314,1438.162,847.529,286.406,10.196,14.955,22.558,0.613,0.113
HR000315,1438.162,551.788,45.239,8.578,23.220,16.555,0.636,0.156


# Data export

In [26]:
# Export the final dataset:
topograhpy_df.to_csv(PATH_OUTPUT+"estreams_topography_attributes.csv")

# End